![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [92]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import RandomizedSearchCV

rental_df = pd.read_csv("rental_info.csv")
rental_df["rental_date"] = pd.to_datetime(rental_df["rental_date"])
rental_df["return_date"] = pd.to_datetime(rental_df["return_date"])
rental_df["rental_length_days"] = (rental_df["return_date"] - rental_df["rental_date"]).dt.days

rental_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   rental_date         15861 non-null  datetime64[ns, UTC]
 1   return_date         15861 non-null  datetime64[ns, UTC]
 2   amount              15861 non-null  float64            
 3   release_year        15861 non-null  float64            
 4   rental_rate         15861 non-null  float64            
 5   length              15861 non-null  float64            
 6   replacement_cost    15861 non-null  float64            
 7   special_features    15861 non-null  object             
 8   NC-17               15861 non-null  int64              
 9   PG                  15861 non-null  int64              
 10  PG-13               15861 non-null  int64              
 11  R                   15861 non-null  int64              
 12  amount_2            15861 non-nu

In [93]:
# Creating two columns of dummy variables from "special_features"
rental_df['deleted_scenes'] = np.where(rental_df['special_features'].str.contains("Deleted Scenes"), 1, 0)
rental_df['behind_the_scenes'] = np.where(rental_df['special_features'].str.contains("Behind the Scenes"), 1, 0)

y = rental_df['rental_length_days']
X = rental_df.drop(['rental_date', 'return_date', 'special_features', 'rental_length_days'], axis=1)

X.head()

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,deleted_scenes,behind_the_scenes
0,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
1,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
2,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
3,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1
4,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,0,1


In [94]:
# Performing feature selection using Lasso

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

lasso = Lasso(alpha = 0.5, random_state=9)
lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_
X_train_lasso, X_test_lasso = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]
X_train.head()

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,deleted_scenes,behind_the_scenes
6682,2.99,2010.0,2.99,90.0,25.99,1,0,0,0,8.9401,8100.0,8.9401,0,1
8908,4.99,2008.0,0.99,53.0,25.99,1,0,0,0,24.9001,2809.0,0.9801,1,0
11827,6.99,2007.0,4.99,171.0,25.99,0,0,1,0,48.8601,29241.0,24.9001,0,1
6153,2.99,2010.0,2.99,73.0,29.99,0,0,0,1,8.9401,5329.0,8.9401,1,1
10713,5.99,2004.0,0.99,122.0,14.99,1,0,0,0,35.8801,14884.0,0.9801,1,0


In [95]:
# Choosing models

models = [LinearRegression(), DecisionTreeRegressor(), RandomForestRegressor()]
mse = list()

for model in models:
    model.fit(X_train, y_train)
    mse.append(MSE(y_test, model.predict(X_test)))
print(mse)

# Here I can see that RandomForestRegressor performs better using MSE as metric

[2.9417238646975883, 2.1729190942279, 2.026046150252537]


In [96]:
# Tuning RandomForestRegressor model
param_dist = {"n_estimators": np.arange(1, 101, 1), "max_depth" : np.arange(1,11,1)}
cv = RandomizedSearchCV(RandomForestRegressor(), param_dist, cv=5, random_state=9, verbose=1)
cv.fit(X_train, y_train)
best_model = cv.best_estimator_

best_model

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomForestRegressor(max_depth=10, n_estimators=51)

In [97]:
# Predicting values on test set

y_pred = best_model.predict(X_test)
best_mse = MSE(y_test, y_pred)
print(best_mse)

2.221732036028172
